<a href="https://colab.research.google.com/github/mcharan/NaiveBayesArticle/blob/main/PUCPR_MestradoPPGIA_ArtigoEstat%C3%ADstica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artigo - Implementação do Naive Bayes

# Imports e carga dos dados

In [14]:
import pandas as pd
import numpy as np
import math
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from scipy import stats
import tracemalloc
import time

In [2]:
data = pd.read_excel("app_data.xlsx", sheet_name="All cases")
data = data.dropna(how='all')

## Pré-processamento

In [3]:
len(data)

781

In [4]:
label_columns = ['Management', 'Severity', 'Diagnosis']
classes = data[label_columns].copy()
features = data.drop(columns=label_columns, errors='ignore')
features = features.drop(columns=['Diagnosis_Presumptive'], errors='ignore')


In [5]:
print("Shape das features:", features.shape)
print("Shape dos labels:", classes.shape)
print("Colunas das features:", len(features.columns))

Shape das features: (781, 54)
Shape dos labels: (781, 3)
Colunas das features: 54


In [6]:
print("Tipos das features:")
print(features.dtypes.value_counts())

Tipos das features:
object     36
float64    18
Name: count, dtype: int64


## Implementação própria do Algoritmo Naive Bayes

In [15]:
class NaiveBayesSimples:
    def __init__(self):
        self.probabilidades_classe = {}  # P(classe)
        self.probabilidades_categoricas = {}  # P(feature|classe) para categóricas
        self.estatisticas_numericas = {}  # média e desvio para numéricas
        self.features_categoricas = []
        self.features_numericas = []
        self.classes = []

    def identificar_tipos_features(self, features):
        """
        Identifica quais features são categóricas e quais são numéricas
        """
        categoricas = []
        numericas = []

        for coluna in features.columns:
            if features[coluna].dtype in ['object', 'category']:
                categoricas.append(coluna)
            elif features[coluna].dtype in ['int64', 'float64']:
                # Se tem poucos valores únicos, pode ser categórica
                valores_unicos = features[coluna].nunique()
                #print(f"Coluna {coluna}, numérica, tem {valores_unicos} valores únicos.")
                if valores_unicos <= 12:  # threshold simples
                    categoricas.append(coluna)
                    #print(f"Coluna {coluna}, do tipo {features[coluna].dtype}, tem {valores_unicos} valores únicos e portanto será considerada categórica.")
                else:
                    numericas.append(coluna)

        #print(f"Features categóricas: {len(categoricas)}")
        #print(f"Features numéricas: {len(numericas)}")

        return categoricas, numericas

    def calcular_probabilidades_classe(self, y):
        """
        Calcula P(classe) para cada classe
        """
        total = len(y)
        classes_unicas = y.value_counts()

        probabilidades = {}
        for classe, count in classes_unicas.items():
            probabilidades[classe] = count / total

        #print("Probabilidades das classes:")
        #for classe, prob in probabilidades.items():
        #    print(f"  P({classe}) = {prob:.4f}")

        return probabilidades

    def calcular_probabilidades_categoricas(self, features, y):
        """
        Calcula P(feature_valor|classe) para features categóricas
        """
        probabilidades = {}

        for feature in self.features_categoricas:
            probabilidades[feature] = {}

            for classe in self.classes:
                # Dados da classe específica
                dados_classe = features[features.index.isin(y[y == classe].index)]
                valores_feature = dados_classe[feature]

                # Contar valores (incluindo NaN)
                counts = valores_feature.value_counts(dropna=False)
                total_classe = len(valores_feature)

                probabilidades[feature][classe] = {}

                # Calcular probabilidade para cada valor (com suavização de Laplace)
                valores_unicos = features[feature].value_counts(dropna=False).index

                for valor in valores_unicos:
                    count_valor = counts.get(valor, 0)
                    # Suavização de Laplace: +1 no numerador, +n_valores no denominador
                    prob = (count_valor + 1) / (total_classe + len(valores_unicos))
                    probabilidades[feature][classe][valor] = prob

        return probabilidades

    def calcular_estatisticas_numericas(self, features, y):
        """
        Calcula média e desvio padrão para features numéricas por classe
        """
        estatisticas = {}

        for feature in self.features_numericas:
            estatisticas[feature] = {}

            for classe in self.classes:
                # Dados da classe específica
                dados_classe = features[features.index.isin(y[y == classe].index)]
                valores_feature = dados_classe[feature].dropna()  # Remove NaN

                if len(valores_feature) > 0:
                    media = valores_feature.mean()
                    desvio = valores_feature.std()

                    # Evitar desvio zero (adicionar pequeno valor)
                    if desvio == 0:
                        desvio = 1e-6

                    estatisticas[feature][classe] = {
                        'media': media,
                        'desvio': desvio
                    }
                else:
                    # Se não há dados, usar valores padrão
                    estatisticas[feature][classe] = {
                        'media': 0,
                        'desvio': 1
                    }

        return estatisticas

    def probabilidade_normal(self, x, media, desvio):
        """
        Calcula probabilidade usando distribuição normal
        """
        if pd.isna(x):
            return 1.0  # Se for NaN, assumir probabilidade neutra

        expoente = -((x - media) ** 2) / (2 * desvio ** 2)
        return (1 / (math.sqrt(2 * math.pi) * desvio)) * math.exp(expoente)

    def treinar(self, features, y):
        """
        Treina o modelo Naive Bayes
        """
        #print("=== TREINANDO NAIVE BAYES ===")

        # Identificar tipos de features
        self.features_categoricas, self.features_numericas = self.identificar_tipos_features(features)

        # Obter classes únicas
        self.classes = y.unique()
        #print(f"Classes encontradas: {self.classes}")

        # Calcular probabilidades das classes
        self.probabilidades_classe = self.calcular_probabilidades_classe(y)

        # Calcular probabilidades para features categóricas
        if self.features_categoricas:
            #print("\nCalculando probabilidades categóricas...")
            self.probabilidades_categoricas = self.calcular_probabilidades_categoricas(features, y)

        # Calcular estatísticas para features numéricas
        if self.features_numericas:
            #print("Calculando estatísticas numéricas...")
            self.estatisticas_numericas = self.calcular_estatisticas_numericas(features, y)

        #print("Treinamento concluído!")

    def predizer_um_exemplo(self, exemplo):
        """
        Prediz a classe para um único exemplo
        """
        probabilidades_finais = {}

        for classe in self.classes:
            # Começar com probabilidade a priori da classe
            prob_classe = self.probabilidades_classe[classe]

            # Multiplicar pelas probabilidades das features categóricas
            for feature in self.features_categoricas:
                if feature in exemplo.index:
                    valor = exemplo[feature]

                    if valor in self.probabilidades_categoricas[feature][classe]:
                        prob_feature = self.probabilidades_categoricas[feature][classe][valor]
                    else:
                        # Valor não visto no treinamento (suavização)
                        prob_feature = 1 / (len(self.probabilidades_categoricas[feature][classe]) + 1)

                    prob_classe *= prob_feature

            # Multiplicar pelas probabilidades das features numéricas
            for feature in self.features_numericas:
                if feature in exemplo.index:
                    valor = exemplo[feature]

                    if not pd.isna(valor):
                        media = self.estatisticas_numericas[feature][classe]['media']
                        desvio = self.estatisticas_numericas[feature][classe]['desvio']
                        prob_feature = self.probabilidade_normal(valor, media, desvio)
                        prob_classe *= prob_feature

            probabilidades_finais[classe] = prob_classe

        # Retornar a classe com maior probabilidade
        classe_predita = max(probabilidades_finais, key=probabilidades_finais.get)
        return classe_predita, probabilidades_finais

    def predizer(self, features):
        """
        Prediz classes para múltiplos exemplos
        """
        predicoes = []

        for idx, exemplo in features.iterrows():
            classe_predita, _ = self.predizer_um_exemplo(exemplo)
            predicoes.append(classe_predita)

        return predicoes

    def avaliar(self, features, y_true):
        """
        Avalia o modelo calculando acurácia
        """
        predicoes = self.predizer(features)
        acertos = sum(p == t for p, t in zip(predicoes, y_true))
        acuracia = acertos / len(y_true)

        print(f"Acurácia: {acuracia:.4f} ({acertos}/{len(y_true)})")

        return acuracia, predicoes



## Testes iniciais com a implementação própria

In [10]:
# Remover registro com NaN no label 'Diagnosis'
mask_valido = classes['Diagnosis'].notna()
features_limpo = features[mask_valido]
classes_limpo = classes[mask_valido]

print(f"Registros antes: {len(features)}")
print(f"Registros depois: {len(features_limpo)}")
print(f"Registros removidos: {len(features) - len(features_limpo)}")
classes_limpo['Diagnosis'].value_counts()

Registros antes: 781
Registros depois: 780
Registros removidos: 1


,count
Diagnosis,
appendicitis,463
no appendicitis,317


In [11]:
modelo = NaiveBayesSimples()

# Escolher um label para treinar (por exemplo, 'Diagnosis')
y_treino = classes_limpo['Diagnosis']

# Treinar o modelo
modelo.treinar(features_limpo, y_treino)

# Fazer predições
#predicoes = modelo.predizer(features_limpo)

# Avaliar
acuracia, predicoes = modelo.avaliar(features_limpo, y_treino)


Acurácia: 0.8474 (661/780)


In [12]:
modelo.features_categoricas

['Sex',
 'Alvarado_Score',
 'Paedriatic_Appendicitis_Score',
 'Appendix_on_US',
 'Migratory_Pain',
 'Lower_Right_Abd_Pain',
 'Contralateral_Rebound_Tenderness',
 'Coughing_Pain',
 'Nausea',
 'Loss_of_Appetite',
 'Neutrophilia',
 'Ketones_in_Urine',
 'RBC_in_Urine',
 'WBC_in_Urine',
 'Dysuria',
 'Stool',
 'Peritonitis',
 'Psoas_Sign',
 'Ipsilateral_Rebound_Tenderness',
 'US_Performed',
 'Free_Fluids',
 'Appendix_Wall_Layers',
 'Target_Sign',
 'Appendicolith',
 'Perfusion',
 'Perforation',
 'Surrounding_Tissue_Reaction',
 'Appendicular_Abscess',
 'Abscess_Location',
 'Pathological_Lymph_Nodes',
 'Lymph_Nodes_Location',
 'Bowel_Wall_Thickening',
 'Conglomerate_of_Bowel_Loops',
 'Ileus',
 'Coprostasis',
 'Meteorism',
 'Enteritis',
 'Gynecological_Findings']

## Esboços de funções para o protocolo

In [16]:
def preparar_dados_treino_validacao(features, classes, nome_classe='Diagnosis', test_size=0.2, random_state=42, stratify=True):
    """
    Prepara dados para treino e validação

    Args:
        features: DataFrame com features
        classes: DataFrame com classes
        nome_classe: Nome da classe a ser usada
        test_size: Proporção para validação (0.2 = 20%)
        random_state: Seed para reprodutibilidade
        stratify: Se True, mantém a proporção das classes nas divisões
    """

    print(f"=== PREPARANDO DADOS PARA TREINO E VALIDAÇÃO ===")

    # 1. Remover registros com NaN na classe escolhida
    mask_valido = classes[nome_classe].notna()
    features_limpo = features[mask_valido]
    y_limpo = classes[nome_classe][mask_valido]

    print(f"Registros antes da limpeza: {len(features)}")
    print(f"Registros após limpeza: {len(features_limpo)}")
    print(f"Registros removidos: {len(features) - len(features_limpo)}")

    # 2. Mostrar distribuição das classes
    print(f"\nDistribuição da classe '{nome_classe}':")
    distribuicao = y_limpo.value_counts().sort_index()
    for classe, count in distribuicao.items():
        percentual = (count / len(y_limpo)) * 100
        print(f"  {classe}: {count} ({percentual:.1f}%)")

    # 3. Dividir dados em treino e validação
    X_treino, X_validacao, y_treino, y_validacao = train_test_split(
        features_limpo,
        y_limpo,
        test_size=test_size,
        random_state=random_state,
        stratify=y_limpo if stratify else None  # Manter proporção das classes
    )

    print(f"\n=== DIVISÃO DOS DADOS ===")
    print(f"Conjunto de treino: {len(X_treino)} exemplos ({(1-test_size)*100:.0f}%)")
    print(f"Conjunto de validação: {len(X_validacao)} exemplos ({test_size*100:.0f}%)")

    # 4. Verificar distribuição após divisão
    print(f"\nDistribuição no TREINO:")
    dist_treino = y_treino.value_counts().sort_index()
    for classe, count in dist_treino.items():
        percentual = (count / len(y_treino)) * 100
        print(f"  {classe}: {count} ({percentual:.1f}%)")

    print(f"\nDistribuição na VALIDAÇÃO:")
    dist_validacao = y_validacao.value_counts().sort_index()
    for classe, count in dist_validacao.items():
        percentual = (count / len(y_validacao)) * 100
        print(f"  {classe}: {count} ({percentual:.1f}%)")

    return X_treino, X_validacao, y_treino, y_validacao

def treinar_e_validar_modelo(X_treino, X_validacao, y_treino, y_validacao):
    """
    Treina o modelo e valida
    """
    print(f"\n=== TREINANDO MODELO ===")

    # Criar e treinar modelo
    modelo = NaiveBayesSimples()
    modelo.treinar(X_treino, y_treino)

    print(f"\n=== AVALIANDO NO TREINO ===")
    acuracia_treino, pred_treino = modelo.avaliar(X_treino, y_treino)

    print(f"\n=== AVALIANDO NA VALIDAÇÃO ===")
    acuracia_validacao, pred_validacao = modelo.avaliar(X_validacao, y_validacao)

    # Comparar resultados
    print(f"\n=== RESUMO DOS RESULTADOS ===")
    print(f"Acurácia no treino: {acuracia_treino:.4f}")
    print(f"Acurácia na validação: {acuracia_validacao:.4f}")

    diferenca = acuracia_treino - acuracia_validacao
    print(f"Diferença: {diferenca:.4f}")

    if diferenca > 0.1:
        print("⚠️  Possível overfitting (diferença > 10%)")
    elif diferenca < 0.05:
        print("✅ Bom balanceamento entre treino e validação")
    else:
        print("📊 Diferença moderada entre treino e validação")

    return modelo, acuracia_treino, acuracia_validacao



In [18]:
# 1. Preparar dados
X_treino, X_validacao, y_treino, y_validacao = preparar_dados_treino_validacao(
    features,
    classes,
    nome_classe='Diagnosis',  # ou 'Management', 'Severity'
    test_size=0.2,  # 20% para validação
    random_state=42,
    stratify=False
)

# 2. Treinar e validar
modelo, acc_treino, acc_validacao = treinar_e_validar_modelo(
    X_treino, X_validacao, y_treino, y_validacao
)

# 3. Usar modelo para fazer predições
# nova_predicao = modelo.predizer(novo_exemplo)




=== PREPARANDO DADOS PARA TREINO E VALIDAÇÃO ===
Registros antes da limpeza: 781
Registros após limpeza: 780
Registros removidos: 1

Distribuição da classe 'Diagnosis':
  appendicitis: 463 (59.4%)
  no appendicitis: 317 (40.6%)

=== DIVISÃO DOS DADOS ===
Conjunto de treino: 624 exemplos (80%)
Conjunto de validação: 156 exemplos (20%)

Distribuição no TREINO:
  appendicitis: 372 (59.6%)
  no appendicitis: 252 (40.4%)

Distribuição na VALIDAÇÃO:
  appendicitis: 91 (58.3%)
  no appendicitis: 65 (41.7%)

=== TREINANDO MODELO ===

=== AVALIANDO NO TREINO ===
Acurácia: 0.8478 (529/624)

=== AVALIANDO NA VALIDAÇÃO ===
Acurácia: 0.8590 (134/156)

=== RESUMO DOS RESULTADOS ===
Acurácia no treino: 0.8478
Acurácia na validação: 0.8590
Diferença: -0.0112
✅ Bom balanceamento entre treino e validação


In [19]:
# Função adicional para testar diferentes proporções
def testar_diferentes_proporcoes(features, classes, nome_classe='Diagnosis', stratify=True):
    """
    Testa diferentes proporções de treino/validação
    """
    proporcoes = [0.1, 0.2, 0.3]

    print(f"=== TESTANDO DIFERENTES PROPORÇÕES ===")

    for prop in proporcoes:
        print(f"\n--- Proporção validação: {prop*100:.0f}% ---")

        X_treino, X_validacao, y_treino, y_validacao = preparar_dados_treino_validacao(
            features, classes, nome_classe, test_size=prop, random_state=42, stratify=stratify
        )

        modelo, acc_treino, acc_validacao = treinar_e_validar_modelo(
            X_treino, X_validacao, y_treino, y_validacao
        )

        print(f"Treino: {acc_treino:.4f} | Validação: {acc_validacao:.4f}")
        print("=="*40)

testar_diferentes_proporcoes(features, classes,nome_classe='Diagnosis', stratify=False)

=== TESTANDO DIFERENTES PROPORÇÕES ===

--- Proporção validação: 10% ---
=== PREPARANDO DADOS PARA TREINO E VALIDAÇÃO ===
Registros antes da limpeza: 781
Registros após limpeza: 780
Registros removidos: 1

Distribuição da classe 'Diagnosis':
  appendicitis: 463 (59.4%)
  no appendicitis: 317 (40.6%)

=== DIVISÃO DOS DADOS ===
Conjunto de treino: 702 exemplos (90%)
Conjunto de validação: 78 exemplos (10%)

Distribuição no TREINO:
  appendicitis: 416 (59.3%)
  no appendicitis: 286 (40.7%)

Distribuição na VALIDAÇÃO:
  appendicitis: 47 (60.3%)
  no appendicitis: 31 (39.7%)

=== TREINANDO MODELO ===

=== AVALIANDO NO TREINO ===
Acurácia: 0.8476 (595/702)

=== AVALIANDO NA VALIDAÇÃO ===
Acurácia: 0.8846 (69/78)

=== RESUMO DOS RESULTADOS ===
Acurácia no treino: 0.8476
Acurácia na validação: 0.8846
Diferença: -0.0370
✅ Bom balanceamento entre treino e validação
Treino: 0.8476 | Validação: 0.8846

--- Proporção validação: 20% ---
=== PREPARANDO DADOS PARA TREINO E VALIDAÇÃO ===
Registros ante

## Implementação do Protocolo de Testes

In [21]:
import warnings
warnings.filterwarnings('ignore')


def executar_teste_unico(features, classes, nome_classe, test_size, random_state, implementacao='proprio'):
    """
    Executa um único teste e retorna métricas completas
    """
    try:
        # Preparar dados
        mask_valido = classes[nome_classe].notna()
        features_limpo = features[mask_valido]
        y_limpo = classes[nome_classe][mask_valido]

        X_treino, X_validacao, y_treino, y_validacao = train_test_split(
            features_limpo, y_limpo, test_size=test_size,
            random_state=random_state, stratify=y_limpo
        )

        memoria_treino_kb = 0
        memoria_pred_kb = 0

        if implementacao == 'proprio':

            modelo = NaiveBayesSimples()

            # --- Medição do Treinamento ---
            tracemalloc.start()
            inicio_treino = time.time()

            modelo.treinar(X_treino, y_treino)

            tempo_treino = time.time() - inicio_treino
            _, pico_memoria = tracemalloc.get_traced_memory() # Retorna (atual, pico) em bytes
            tracemalloc.stop()
            memoria_treino_kb = pico_memoria / 1024 # Convertendo para KB

            # --- Medição da Predição ---
            tracemalloc.start()
            inicio_pred = time.time()

            pred_validacao = modelo.predizer(X_validacao)

            tempo_predicao = time.time() - inicio_pred
            _, pico_memoria = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            memoria_pred_kb = pico_memoria / 1024

            acuracia = sum(p == t for p, t in zip(pred_validacao, y_validacao)) / len(y_validacao)

        else:  # sklearn
            # Preparar dados para sklearn
            X_treino_sklearn, X_validacao_sklearn = preparar_dados_sklearn_simples(X_treino, X_validacao)
            modelo = GaussianNB()

            # --- Medição do Treinamento ---
            tracemalloc.start()
            inicio_treino = time.time()

            modelo.fit(X_treino_sklearn, y_treino)

            tempo_treino = time.time() - inicio_treino
            _, pico_memoria = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            memoria_treino_kb = pico_memoria / 1024

            # --- Medição da Predição ---
            tracemalloc.start()
            inicio_pred = time.time()

            pred_validacao = modelo.predict(X_validacao_sklearn)

            tempo_predicao = time.time() - inicio_pred
            _, pico_memoria = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            memoria_pred_kb = pico_memoria / 1024

            acuracia = (pred_validacao == y_validacao).mean()

        return {
            'acuracia': acuracia,
            'tempo_treino': tempo_treino,
            'tempo_predicao': tempo_predicao,
            'memoria_treino_kb': memoria_treino_kb, # Nova métrica
            'memoria_pred_kb': memoria_pred_kb,   # Nova métrica
            'n_treino': len(X_treino),
            'n_validacao': len(X_validacao),
            'sucesso': True
        }

    except Exception as e:
        return {
            'acuracia': np.nan,
            'tempo_treino': np.nan,
            'tempo_predicao': np.nan,
            'memoria_treino_kb': np.nan,
            'memoria_pred_kb': np.nan,
            'n_treino': np.nan,
            'n_validacao': np.nan,
            'sucesso': False,
            'erro': str(e)
        }

def preparar_dados_sklearn_simples(X_treino, X_validacao):
    """Versão simplificada da preparação para sklearn"""
    X_treino_sklearn = X_treino.copy()
    X_validacao_sklearn = X_validacao.copy()

    # Identificar e codificar categóricas
    colunas_categoricas = []
    for coluna in X_treino.columns:
        if X_treino[coluna].dtype in ['object', 'category'] or X_treino[coluna].nunique() <= 10:
            colunas_categoricas.append(coluna)

     # 1. Tratar NaNs nas colunas categóricas ANTES de codificar
    for coluna in colunas_categoricas:
        placeholder = 'missing' # Usar um placeholder consistente
        X_treino_sklearn[coluna] = X_treino_sklearn[coluna].fillna(placeholder)
        X_validacao_sklearn[coluna] = X_validacao_sklearn[coluna].fillna(placeholder)

    # 2. Codificar as colunas categóricas
    for coluna in colunas_categoricas:
        le = LabelEncoder()
        # Agora não precisamos mais do .dropna()
        valores_combinados = pd.concat([X_treino_sklearn[coluna], X_validacao_sklearn[coluna]]).astype(str)

        le.fit(valores_combinados)
        X_treino_sklearn[coluna] = le.transform(X_treino_sklearn[coluna].astype(str))
        X_validacao_sklearn[coluna] = le.transform(X_validacao_sklearn[coluna].astype(str))

    # 3. Tratar NaNs nas colunas numéricas restantes
    X_treino_sklearn = X_treino_sklearn.fillna(-1)
    X_validacao_sklearn = X_validacao_sklearn.fillna(-1)

    return X_treino_sklearn, X_validacao_sklearn

def protocolo_teste_completo(features, classes, nome_classe,
                           sementes=[42, 123, 456, 789, 101112, 131415, 161718, 192021, 222324, 252627],
                           test_sizes=[0.1, 0.2, 0.3],
                           implementacoes=['proprio', 'sklearn']):
    """
    Executa protocolo completo de testes
    """
    print(f"=== PROTOCOLO DE TESTE PARA CLASSE: {nome_classe} ===")
    print(f"Sementes: {len(sementes)} | Test sizes: {test_sizes} | Implementações: {implementacoes}")

    resultados = []

    total_testes = len(sementes) * len(test_sizes) * len(implementacoes)
    contador = 0

    for implementacao in implementacoes:
        for test_size in test_sizes:
            for semente in sementes:
                contador += 1
                print(f"Teste {contador}/{total_testes}: {implementacao} - {test_size*100}% - seed {semente}", end="")

                resultado = executar_teste_unico(features, classes, nome_classe, test_size, semente, implementacao)

                # Adicionar metadados
                resultado.update({
                    'implementacao': implementacao,
                    'test_size': test_size,
                    'semente': semente,
                    'nome_classe': nome_classe
                })

                resultados.append(resultado)

                if resultado['sucesso']:
                    print(f" ✓ Acc: {resultado['acuracia']:.4f}")
                else:
                    print(f" ✗ ERRO: {resultado.get('erro', 'Desconhecido')}")

    return pd.DataFrame(resultados)

def analisar_resultados_estatisticos(df_resultados):
    """
    Análise estatística completa dos resultados
    """
    print("\n=== ANÁLISE ESTATÍSTICA DOS RESULTADOS ===")

    # Filtrar apenas sucessos
    df_validos = df_resultados[df_resultados['sucesso'] == True].copy()

    if len(df_validos) < 2:
        print("❌ Dados insuficientes para análise estatística!")
        return None

    print(f"Testes válidos: {len(df_validos)}/{len(df_resultados)}")

    # 1. Estatísticas descritivas por implementação
    print("\n=== ESTATÍSTICAS DESCRITIVAS ===")

    metricas = ['acuracia', 'tempo_treino', 'tempo_predicao', 'memoria_treino_kb', 'memoria_pred_kb']
    stats_desc = df_validos.groupby('implementacao')[metricas].agg(['mean', 'std', 'min', 'max']).round(6)

    print(stats_desc)

    # 2. Testes de hipóteses
    print("\n=== TESTES DE HIPÓTESES PAREADOS ===")

    implementacoes = df_validos['implementacao'].unique()

    if len(implementacoes) == 2:
        impl1, impl2 = implementacoes[0], implementacoes[1]

        resultados_testes = {}

        for metrica in metricas:

            # Pivotar a tabela para alinhar os pares
            df_pivot = df_validos.pivot_table(
                index=['semente', 'test_size'],
                columns='implementacao',
                values=metrica
            ).dropna()

            if len(df_pivot) == 0:
                print(f"\n{metrica.upper()}: Sem pares completos para análise.")
                continue

            dados1 = df_pivot[impl1]
            dados2 = df_pivot[impl2]

            # Teste de normalidade sobre as DIFERENÇAS entre os pares
            diferencas = dados1 - dados2
            _, p_norm = stats.shapiro(diferencas)

            if p_norm > 0.05:
                # Dados normais - usar Teste T Pareado (ttest_rel)
                stat, p_value = stats.ttest_rel(dados1, dados2)
                teste_usado = "Teste T Pareado"
            else:
                # Dados não normais - usar Teste de Wilcoxon
                # Nota: Wilcoxon não lida bem com diferenças zero. `zero_method='zsplit'` é uma abordagem.
                try:
                    stat, p_value = stats.wilcoxon(dados1, dados2, zero_method='zsplit')
                    teste_usado = "Wilcoxon"
                except ValueError:
                     # Ocorre se todas as diferenças forem zero
                    stat, p_value = 0, 1.0
                    teste_usado = "Wilcoxon (sem diferenças)"

            # Calcular tamanho do efeito (Cohen's d) para amostras pareadas
            # d = média das diferenças / desvio padrão das diferenças
            mean_diff = np.mean(diferencas)
            std_diff = np.std(diferencas, ddof=1)
            cohen_d = mean_diff / std_diff if std_diff > 0 else 0

            # Interpretar tamanho do efeito
            if abs(cohen_d) < 0.2:
                efeito = "Pequeno"
            elif abs(cohen_d) < 0.5:
                efeito = "Médio"
            else:
                efeito = "Grande"

            resultados_testes[metrica] = {
                'teste': teste_usado,
                'estatistica': stat,
                'p_value': p_value,
                'significativo': p_value < 0.05,
                'cohen_d': cohen_d,
                'tamanho_efeito': efeito
            }

            print(f"\n{metrica.upper()}:")
            print(f"  Comparando {len(df_pivot)} pares.")
            print(f"  {impl1}: μ={dados1.mean():.6f}, σ={dados1.std():.6f}")
            print(f"  {impl2}: μ={dados2.mean():.6f}, σ={dados2.std():.6f}")
            print(f"  Teste: {teste_usado} (baseado na normalidade das diferenças)")
            print(f"  p-value: {p_value:.6f} {'***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns'}")
            print(f"  Cohen's d (pareado): {cohen_d:.4f} ({efeito})")

        return resultados_testes

    else:
        print("⚠️  Análise comparativa requer exatamente 2 implementações")
        return None

def executar_protocolo_multiplas_classes(features, classes,
                                       classes_teste=['Diagnosis', 'Management', 'Severity'],
                                       sementes=None, test_sizes=None):
    """
    Executa protocolo para múltiplas classes
    """
    if sementes is None:
        sementes = [42, 123, 456, 789, 101112, 131415, 161718, 192021, 222324, 252627]

    if test_sizes is None:
        test_sizes = [0.1, 0.2, 0.3]

    print("="*80)
    print("PROTOCOLO DE TESTE ESTATÍSTICO - MÚLTIPLAS CLASSES")
    print("="*80)

    resultados_por_classe = {}
    testes_estatisticos = {}

    for nome_classe in classes_teste:
        print(f"\n{'='*20} PROCESSANDO CLASSE: {nome_classe} {'='*20}")

        try:
            # Executar protocolo
            df_resultados = protocolo_teste_completo(
                features, classes, nome_classe,
                sementes=sementes, test_sizes=test_sizes
            )

            # Análise estatística
            testes_stats = analisar_resultados_estatisticos(df_resultados)

            resultados_por_classe[nome_classe] = df_resultados
            testes_estatisticos[nome_classe] = testes_stats

        except Exception as e:
            print(f"❌ Erro ao processar classe {nome_classe}: {str(e)}")
            resultados_por_classe[nome_classe] = None
            testes_estatisticos[nome_classe] = None

    # Resumo final
    print("\n" + "="*80)
    print("RESUMO FINAL - SIGNIFICÂNCIA ESTATÍSTICA")
    print("="*80)

    for classe in classes_teste:
        print(f"\n=== {classe} ===")
        if testes_estatisticos[classe]:
            for metrica, resultado in testes_estatisticos[classe].items():
                significancia = "***" if resultado['p_value'] < 0.001 else \
                              "**" if resultado['p_value'] < 0.01 else \
                              "*" if resultado['p_value'] < 0.05 else "ns"

                print(f"{metrica:<20}: p={resultado['p_value']:.6f} {significancia} (Cohen's d={resultado['cohen_d']:.4f})")
        else:
            print("Dados não disponíveis")

    return resultados_por_classe, testes_estatisticos

In [22]:

# Teste completo para uma classe
df_resultados = protocolo_teste_completo(features, classes, 'Diagnosis')
testes_stats = analisar_resultados_estatisticos(df_resultados)

# Teste para múltiplas classes
#resultados_todas, testes_todas = executar_protocolo_multiplas_classes(
#    features, classes,
#    classes_teste=['Diagnosis', 'Management', 'Severity']
#)

# Salvar resultados
#for classe, df in resultados_todas.items():
#    if df is not None:
#        df.to_csv(f'resultados_{classe}.csv', index=False)


=== PROTOCOLO DE TESTE PARA CLASSE: Diagnosis ===
Sementes: 10 | Test sizes: [0.1, 0.2, 0.3] | Implementações: ['proprio', 'sklearn']
Teste 1/60: proprio - 10.0% - seed 42 ✓ Acc: 0.8462
Teste 2/60: proprio - 10.0% - seed 123 ✓ Acc: 0.8205
Teste 3/60: proprio - 10.0% - seed 456 ✓ Acc: 0.8333
Teste 4/60: proprio - 10.0% - seed 789 ✓ Acc: 0.8077
Teste 5/60: proprio - 10.0% - seed 101112 ✓ Acc: 0.8974
Teste 6/60: proprio - 10.0% - seed 131415 ✓ Acc: 0.8077
Teste 7/60: proprio - 10.0% - seed 161718 ✓ Acc: 0.7821
Teste 8/60: proprio - 10.0% - seed 192021 ✓ Acc: 0.8077
Teste 9/60: proprio - 10.0% - seed 222324 ✓ Acc: 0.7821
Teste 10/60: proprio - 10.0% - seed 252627 ✓ Acc: 0.8718
Teste 11/60: proprio - 20.0% - seed 42 ✓ Acc: 0.8397
Teste 12/60: proprio - 20.0% - seed 123 ✓ Acc: 0.8590
Teste 13/60: proprio - 20.0% - seed 456 ✓ Acc: 0.8141
Teste 14/60: proprio - 20.0% - seed 789 ✓ Acc: 0.8526
Teste 15/60: proprio - 20.0% - seed 101112 ✓ Acc: 0.8718
Teste 16/60: proprio - 20.0% - seed 131415 ✓ A

In [ ]:
df_resultados

,acuracia,tempo_treino,tempo_predicao,memoria_mb,cpu_percent,n_treino,n_validacao,sucesso,implementacao,test_size,semente,nome_classe
0,0.846154,0.116132,0.037099,0.0,0.0,702,78,True,proprio,0.1,42,Diagnosis
1,0.820513,0.119511,0.036159,0.0,0.0,702,78,True,proprio,0.1,123,Diagnosis
2,0.833333,0.150000,0.088684,0.0,0.0,702,78,True,proprio,0.1,456,Diagnosis
3,0.807692,0.285003,0.082452,0.0,0.0,702,78,True,proprio,0.1,789,Diagnosis
4,0.897436,0.219673,0.061770,0.0,0.0,702,78,True,proprio,0.1,101112,Diagnosis
5,0.807692,0.297629,0.089512,0.0,0.0,702,78,True,proprio,0.1,131415,Diagnosis
6,0.782051,0.275566,0.082457,0.0,0.0,702,78,True,proprio,0.1,161718,Diagnosis
7,0.807692,0.242967,0.072169,0.0,0.0,702,78,True,proprio,0.1,192021,Diagnosis
8,0.782051,0.211008,0.105896,0.0,0.0,702,78,True,proprio,0.1,222324,Diagnosis
9,0.871795,0.211481,0.078404,0.0,0.0,702,78,True,proprio,0.1,252627,Diagnosis


In [23]:
testes_stats

{'acuracia': {'teste': 'Wilcoxon',
  'estatistica': np.float64(0.0),
  'p_value': np.float64(1.6986964471794946e-06),
  'significativo': np.True_,
  'cohen_d': np.float64(2.7483723064187626),
  'tamanho_efeito': 'Grande'},
 'tempo_treino': {'teste': 'Wilcoxon',
  'estatistica': np.float64(0.0),
  'p_value': np.float64(1.862645149230957e-09),
  'significativo': np.True_,
  'cohen_d': np.float64(5.778277989907354),
  'tamanho_efeito': 'Grande'},
 'tempo_predicao': {'teste': 'Wilcoxon',
  'estatistica': np.float64(0.0),
  'p_value': np.float64(1.862645149230957e-09),
  'significativo': np.True_,
  'cohen_d': np.float64(2.1468384136902903),
  'tamanho_efeito': 'Grande'},
 'memoria_treino_kb': {'teste': 'Wilcoxon',
  'estatistica': np.float64(0.0),
  'p_value': np.float64(1.862645149230957e-09),
  'significativo': np.True_,
  'cohen_d': np.float64(-4.861724029128272),
  'tamanho_efeito': 'Grande'},
 'memoria_pred_kb': {'teste': 'Wilcoxon',
  'estatistica': np.float64(0.0),
  'p_value': np.f